# Generacion Features de Grafo

## Imports & Load Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import geopandas as gpd
import networkx as nx

import numpy as np
import pandas as pd

from scipy.spatial import cKDTree

import pickle

In [3]:
# Cargar endpoint
infile = open('./data/1_GDB_Layers.pkl','rb')
GDB = pickle.load(infile)
infile.close()

## Adicionar variables de densidad de consumo

In [9]:
UCBT = GDB['UCBT']
UCBT = UCBT.to_crs({'init': 'epsg:31982'})
gUCBT = [x.coords[0] for x in UCBT.geometry]

In [17]:
radio = 50
tree = cKDTree(gUCBT) 

In [18]:
ConCercanos = tree.query_ball_point(gUCBT, radio)

In [15]:
ConCercanos

{50: array([list([0, 20, 30, 155]), list([1]), list([2, 3]), ...,
        list([106143]),
        list([105975, 105986, 105995, 106056, 106057, 106058, 106144]),
        list([106072, 106145])], dtype=object),
 100: array([list([0, 14, 20, 30, 70, 71, 72, 73, 106, 109, 155]), list([1]),
        list([2, 3]), ..., list([106018, 106143]),
        list([105975, 105986, 105995, 106056, 106057, 106058, 106144]),
        list([106072, 106145])], dtype=object),
 250: array([list([0, 14, 20, 25, 30, 33, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 106, 109, 116, 155]),
        list([1, 31]), list([2, 3]), ..., list([106018, 106027, 106143]),
        list([105972, 105975, 105977, 105980, 105986, 105995, 106022, 106056, 106057, 106058, 106059, 106144]),
        list([105994, 106070, 106071, 106072, 106078, 106079, 106080, 106097, 106104, 106138, 106139, 106145])],
       dtype=object)}

In [19]:
UCBT['Cercanos']= ConsCercanos
UCBT['QCercanos']= UCBT['Cercanos'].apply(lambda x: len(x)-1)

In [28]:
UNTRD_QCer = UCBT.groupby('UNI_TR_D').mean().QCercanos.reset_index()

## Exportar exit point

In [29]:
filename = './data/223_ConsumerDensity.pkl'
outfile = open(filename, 'wb')
pickle.dump(UNTRD_QCer, outfile)
outfile.close()